In [1]:
# List all device
from tensorflow.python.client import device_lib
# print(device_lib.list_local_devices())

In [2]:
# Check available GPU
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

Using TensorFlow backend.


['/job:localhost/replica:0/task:0/device:GPU:0']

In [3]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";
# The GPU id to use, usually either "0" or "1";
os.environ["CUDA_VISIBLE_DEVICES"]="0";  

In [4]:
# Importing the libraries
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
from keras.preprocessing.sequence import TimeseriesGenerator
from keras.optimizers import SGD
import math
import pickle

In [5]:
with open("../data/dowjones_calculated/periods.txt", "rb") as fp:   # Unpickling
    dataset = pickle.load(fp)

In [6]:
X_train = dataset[0][0][0].values
y_train = dataset[0][0][1].values
X_test = dataset[1][0][0].values
y_test = dataset[1][0][1].values

In [23]:
X_train = dataset[0][0][0]["AAPL"].values
y_train = dataset[0][0][1]["AAPL"].values
X_test = dataset[1][0][0]["AAPL"].values
y_test = dataset[1][0][1]["AAPL"].values

In [24]:
print(f"x train shape: {X_train.shape}")
print(f"y train shape: {y_train.shape}")
print(f"x test shape: {X_test.shape}")
print(f"y test shape: {y_test.shape}")
# print(f"predicted shape: {predicted.shape}")

x train shape: (750,)
y train shape: (750,)
x test shape: (250,)
y test shape: (250,)


In [25]:
data = X_train
targets = y_train

train_gen = TimeseriesGenerator(data, targets,
                               length=240, sampling_rate=1,
                               batch_size=510)

In [26]:
data = X_test
targets = y_test

test_gen = TimeseriesGenerator(data, targets,
                               length=240, sampling_rate=1,
                               batch_size=250)

In [27]:
X_train = train_gen[0][0]
y_train = train_gen[0][1]
X_test = test_gen[0][0]
y_test = test_gen[0][1]

In [33]:
# Reshaping X_train for efficient modelling
X_train = np.reshape(X_train, (X_train.shape[0],X_train.shape[1], 1))
X_test = np.reshape(X_test, (X_test.shape[0],X_test.shape[1], 1))

In [34]:
X_train.shape

(510, 240, 1)

In [36]:
# expected input data shape: (batch_size, timesteps, data_dim)

# The LSTM architecture
regressor = Sequential()
# First LSTM layer with Dropout regularisation
regressor.add(LSTM(units=25, return_sequences=True, input_shape=(240, 1)))
# The output layer
regressor.add(LSTM(32))
regressor.add(Dense(1, activation='softmax'))
# Compiling the RNN
regressor.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['binary_crossentropy'])

# Fitting to the training set
regressor.fit(X_train,y_train,epochs=2,batch_size=10)

Epoch 1/2
510/510 [==============================] - 19s 38ms/step - loss: 7.6586 - binary_crossentropy: 7.6586
Epoch 2/2
510/510 [==============================] - 18s 35ms/step - loss: 7.6586 - binary_crossentropy: 7.6586


In [21]:
# Evaluating our model
predicted = regressor.predict(X_test)

In [22]:
label = predicted > 0.5
label = label * 1 # Convert boolean to int
label

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0,

In [15]:
(sum(y_test == label)/len(y_test)).mean()

0.4838709677419355

In [16]:
# from keras.models import load_model

# regressor.save('../model/LSTM/my_model.h5')  # creates a HDF5 file 'my_model.h5'
# del regressor  # deletes the existing model

# # returns a compiled model
# # identical to the previous one
# regressor1 = load_model('../model/LSTM/my_model.h5')
# regressor1.predict(X_test)